# **Fact Orders**

In [0]:
df = spark.sql('SELECT * FROM databricks_cat.silver.orders')

### **Replace the columnd customer_id and product_id for the Dimcustomer and Dimproducts**

In [0]:
df_dimcu = spark.sql('SELECT DimCustomerKey, customer_id AS dim_customer_id FROM databricks_cat.gold.dimcustomers')

df_dimpro = spark.sql('SELECT product_id AS DimProductKey, product_id AS dim_product_id FROM databricks_cat.gold.dimproducts')

### **Fact Dataframe**

In [0]:
df_fact = df.join(df_dimcu, df['customer_id'] == df_dimcu['dim_customer_id'], 'left').join(df_dimpro, df['product_id'] == df_dimpro['dim_product_id'], 'left')

df_fact_new = df_fact.drop('dim_customer_id', 'dim_product_id', 'customer_id', 'product_id')

In [0]:
df_fact_new.display()

### **Upsert on Fact Table**

In [0]:
from delta.tables import DeltaTable 

In [0]:
if spark.catalog.tableExists('databricks_cat.gold.FactOrders'):
    dlt_obj = DeltaTable.forName(spark, 'databricks_cat.gold.FactOrders')

    dlt_obj.alias('t')\
        .merge(df_fact_new.alias('s'), 't.order_id = s.order_id AND t.DimCustomerKey = s.DimCustomerKey AND t.DimProductKey = s.DimProductKey')\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()

else:
    df_fact_new.write.format('delta')\
        .option('path', 'abfss://gold@databrickseterjed.dfs.core.windows.net/FactOrders')\
        .saveAsTable('databricks_cat.gold.FactOrders')

In [0]:
%sql
select * from databricks_cat.gold.factorders